In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import random

# Website

In [2]:
website = "https://www.songtexte.de/musiker.html?seite="

In [3]:
weblist = []
for x in range(1,3):
    weblist.append(website+str(x))

In [4]:
# Dict bestehend aus {Seitenzahl:{Künstler:[link],...}...}

In [5]:
songdict = {}
counter = 1
for url in weblist:
    songdict.update({counter:{}})
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    mydivs = soup.find_all("div", {"class": "cover"})
    for c in range(len(mydivs)):
        tag = mydivs[c].a.attrs
        link = tag["href"]
        artist = tag["title"][:-34]
        songdict[counter].update({artist:link})
    counter += 1
    time.sleep(random.random()*1+1)

In [6]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl]},...}...}

In [7]:
artistdict = {}
counter = 0
fail = {}

for element in range(1,len(songdict.keys())+1): #Für jede Seite
    for artist in songdict[element].keys(): #Für jeden Künstler
        counter += 1
        songs = {}
        url = songdict[element][artist]
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9"})[0]
            mydivs = mydivs.find("ul")
            tags = mydivs.find_all("a")
            #print(len(tags))
            for c in range(len(tags)): #Für jeden Song des Künstlers
                song = tags[c]["title"][len(artist)+3:-9]
                link = tags[c]["href"]
                songs.update({song:[link]})
                artistdict.update({artist:songs})
            #print(counter, artist, "erfolgreich")
        except:
            fail.update({artist:mydivs})
            print(counter, artist, "fehlgeschlagen Link:", songdict[element][artist])

        counter += 1
        time.sleep(random.random()*1+1)

In [8]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl,Lyriks],...}...}

In [11]:
backup_artistdict = artistdict.copy()

In [12]:
counter = 0
for artist in artistdict.keys():
    for song in artistdict[artist]:
        counter +=1
        try:
            url = artistdict[artist][song][0]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9 col-xs-12"})
            tag = mydivs[0].find_all("p")[1]
            song_text = str(tag)[23:-22].replace("<br>","").replace("<br/>","")
            artistdict[artist][song].append(song_text)
        except:
            print("Fehler bei", artist, song, url)
        if counter%500 == 0:
            print(counter,"Lieder heruntergeladen")
        time.sleep(random.random()*1+1)

Fehler bei Kygo  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Kygo ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Kygo tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Sam Martin  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsam-martin-2288925.html
Fehler bei Conrad  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fconrad-223752.html
Fehler bei Conrad o //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fconrad-223752.html
Fehler bei Conrad A whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fconrad-223752.html
Fehler bei Lookas  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Flookas-2188874.html
Fehler bei Lookas o //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Flookas-2188874.html
Fehler bei Lookas A whatsapp:

In [38]:
artistdict["Ed Sheeran"]['Thinking Out Loud'][1]

"When your looks don't work like they used to before  And I can't sweep you off of your feet  Will your mouth still remember the taste of my love  Will your eyes still smile from your cheeks   And darling I will be loving you 'til we're 70  And baby my heart could still fall as hard at 23  And I'm thinking 'bout how people fall in love in mysterious ways  Maybe just the touch of a hand  Oh me I fall in love with you every single day  And I just wanna tell you I am   So honey now  Take me into your loving arms  Kiss me under the light of a thousand stars  Place your head on my beating heart  I'm thinking out loud  Maybe we found love right where we are   When my hair's all but gone and my memory fades  And the crowds don't remember my name  When my hands don't play the strings the same way, mm  I know you will still love me the same   'Cause honey your soul can never grow old, it's evergreen  Baby your smile's forever in my mind and memory   I'm thinking 'bout how people fall in love in

In [17]:
def GetGenreByTitle(artist,title):

    url = "https://www.google.com/search?q="
    headers = {'User-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"}
    
    artist = artist.lower()
    artist = artist.replace(" ","+")
    title = title.lower()
    title = title.replace(" ","+")
    
    search = url+artist+"+"+title+"+genre"
    try:
        page = requests.get(search,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        mydivs = soup.find_all("div", {"class":"Z0LcW XcVN5d"})

        if len(mydivs) == 1:
            genre = [str(mydivs)[27:-7]]
        else:
            mydivs = soup.find_all("div", {"class":"rlc__slider-page"})
            genre = []
            for element in range(len(mydivs)):
                genre_title = mydivs[element].find("div", {"class":"title"})
                genre_title = str(genre_title)[19:-6]
                genre.append(genre_title)
    except:
        genre = "Search for Genre failed"
    time.sleep(0.5)
    return genre

In [34]:
#CSV-Datei erstellen
header = ["Artist","Song_title","Song_text","Song_genre","Song_link"]

with open("tracks_001_050.csv", 'w',newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(header)

with open("tracks_001_050.csv", 'a',newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)    
    for artist in artistdict.keys():
        for song in artistdict[artist]:
            if len(artistdict[artist][song]) == 2:
                artistdict[artist][song][1] = artistdict[artist][song][1].replace("\n"," ").replace("\r"," ")
                artistdict[artist][song][1] = artistdict[artist][song][1].replace("&", "and")
                #genre = GetGenreByTitle(artist,song)
                genre = 0
                try:
                    wr.writerow([artist,song,artistdict[artist][song][1],genre,artistdict[artist][song][0]])
                except:
                    print("csv fehlgeschlagen",[artist,song,artistdict[artist][song][1],genre,artistdict[artist][song][0]])
                    print("\n")


csv fehlgeschlagen ['The Weeknd', 'Often', '[Produced by Ben Billions]  [Intro] "Seneler sürer her günüm" Ooh yeah, ooh yeah, ooh yeah, ooh yeah, ooh yeah "Yalnız gitmekten yorgunum" Ooh yeah, ooh yeah, ooh yeah, ooh yeah "Seneler sürer her günüm" Ooh yeah, ooh yeah, ooh yeah, ooh yeah, ooh yeah "Yalnız gitmekten yorgunum" Ooh yeah, ooh yeah, ooh yeah, ooh yeah, ooh yeah  [Verse 1] I usually love sleeping all alone This time around bring your friend with you But we ain\'t really going to sleep at all You ain\'t gonna catch me with them sneak pictures In my city I\'m a young god That pussy kill be so vicious My god white, he in my pocket He get me redder than the devil \'til I go nauseous  [Hook] She asked me if I do this every day, I said "Often" Asked how many times she rode the wave, "Not so often" Bitches down to do it either way, often Baby I can make that pussy rain, often Often, often, girl I do this often Make that pussy pop and do it how I want it Often, often, girl I do this o

In [ ]:
writer = csv.writer(open("songs.csv", "w"))
writer.writerow(header)
writer.writerows(body)

In [29]:
with open("tracks_001_050.csv", 'a',newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(header)
    wr.writerows(body)

UnicodeEncodeError: 'charmap' codec can't encode character '\u0131' in position 141: character maps to <undefined>